# 001 JOIN


```SQL
-- Borramos las tablas si existen para empezar de cero
DROP TABLE IF EXISTS promociones;
DROP TABLE IF EXISTS pedidos;
DROP TABLE IF EXISTS clientes;

-- Tabla de Clientes (Dimensión)
CREATE TABLE clientes (
    cliente_id INTEGER PRIMARY KEY,
    nombre VARCHAR(50),
    ciudad VARCHAR(50)
);

-- Tabla de Pedidos (Hechos)
-- Nota: En un entorno real, cliente_id 99 fallaría si hubiera un FOREIGN KEY estricto,
-- así que la crearemos simple para poder practicar los JOINs libremente.
CREATE TABLE pedidos (
    pedido_id INTEGER PRIMARY KEY,
    cliente_id INTEGER,
    producto VARCHAR(50),
    precio FLOAT
);

-- Tabla de Promociones (Dimensión)
CREATE TABLE promociones (
    ciudad VARCHAR(50),
    descuento VARCHAR(10)
);

-- Insertar Clientes
INSERT INTO clientes (cliente_id, nombre, ciudad)
VALUES
    (1, 'Ana', 'Madrid'),
    (2, 'Luis', 'Bogotá'),
    (3, 'Carla', 'Lima'),
    (4, 'Diego', 'CDMX'),
    (5, 'Elena', 'Santiago');

-- Insertar Pedidos
-- El pedido 104 tiene cliente_id 99 (no existe en la tabla clientes)
INSERT INTO pedidos (pedido_id, cliente_id, producto, precio)
VALUES
    (101, 1, 'Espresso', 3.0),
    (102, 2, 'Latte', 4.5),
    (103, 1, 'Muffin', 2.5),
    (104, 99, 'Capuccino', 5.0),
    (105, 3, 'Té Verde', 3.0);

-- Insertar Promociones
-- Londres está en promociones pero no tenemos clientes allí
INSERT INTO promociones (ciudad, descuento)
VALUES
    ('Madrid', '10%'),
    ('Bogotá', '15%'),
    ('Londres', '20%');

SELECT * FROM clientes;
SELECT * FROM pedidos;
SELECT * FROM promociones;
```

In [1]:
import pandas as pd
import numpy as np

In [3]:
df_clientes = pd.read_csv('Data/001/clientes.csv')
df_pedidos = pd.read_csv('Data/001/pedidos.csv')
df_promociones = pd.read_csv('Data/001/promociones.csv')

df_clientes.head()

,cliente_id,nombre,ciudad
0,1,Ana,Madrid
1,2,Luis,Bogotá
2,3,Carla,Lima
3,4,Diego,CDMX
4,5,Elena,Santiago


In [4]:
df_pedidos.head()

,pedido_id,cliente_id,producto,precio
0,101,1,Espresso,3.0
1,102,2,Latte,4.5
2,103,1,Muffin,2.5
3,104,99,Capuccino,5.0
4,105,3,Té Verde,3.0


In [5]:
df_promociones.head()

,ciudad,descuento
0,Madrid,10%
1,Bogotá,15%
2,Londres,20%


# Pregunta 1: 

### Queremos ver todos los pedidos (incluso el del cliente 99) con el nombre de su cliente.

In [21]:
df_merge = df_pedidos.merge(df_clientes, on='cliente_id', how='left')

respuesta = df_merge[['nombre','producto']]

respuesta

,nombre,producto
0,Ana,Espresso
1,Luis,Latte
2,Ana,Muffin
3,NaN,Capuccino
4,Carla,Té Verde


```SQL
SELECT
    c.nombre,
    pe.producto
FROM pedidos pe
LEFT JOIN clientes c ON  pe.cliente_id = c.cliente_id;
```

# Pregunta 2

### Queremos ver una lista de TODOS los clientes registrados y saber qué han pedido. Si un cliente (como Diego o Elena) no ha pedido nada todavía, su nombre debe aparecer igual.

In [25]:
df_merge2 = df_clientes.merge(df_pedidos, on='cliente_id', how='left')

respuesta2 = df_merge2[['nombre','producto']]

respuesta2

,nombre,producto
0,Ana,Espresso
1,Ana,Muffin
2,Luis,Latte
3,Carla,Té Verde
4,Diego,NaN
5,Elena,NaN


```SQL
SELECT
    c.nombre,
    pe.producto
FROM clientes c
LEFT JOIN pedidos pe ON c.cliente_id = pe.cliente_id
```

# Pregunta 3

### El equipo de Marketing quiere un reporte que muestre: El nombre del cliente, el producto que compró y el descuento que se aplica según su ciudad.

In [28]:
df_paso1 = df_clientes.merge(df_pedidos, on='cliente_id', how='inner')

df_paso = df_paso1.merge(df_promociones, on='ciudad', how='inner')

respuesta3 = df_paso[['nombre','producto','descuento']]

respuesta3

,nombre,producto,descuento
0,Ana,Espresso,10%
1,Ana,Muffin,10%
2,Luis,Latte,15%


```SQL
SELECT
    c.nombre,
    pe.producto,
    pr.descuento
FROM clientes c
INNER JOIN pedidos pe ON c.cliente_id = pe.cliente_id
INNER JOIN promociones pr ON c.ciudad = pr.ciudad;
```

# Pregunta 4

### El dueño de la cafetería quiere saber qué ciudades tenemos registradas en total. Necesita un reporte que muestre todas las ciudades que aparecen en la tabla de clientes Y todas las ciudades que aparecen en la tabla de promociones. Si una ciudad tiene clientes pero no promo, debe aparecer. Si una ciudad tiene promo pero no clientes (como Londres), ¡también debe aparecer!

In [30]:
df_merge4 = df_clientes.merge(df_promociones, on='ciudad', how='outer')

respuseta4 = df_merge4[['ciudad','descuento']]

respuseta4

,ciudad,descuento
0,Bogotá,15%
1,CDMX,NaN
2,Lima,NaN
3,Londres,20%
4,Madrid,10%
5,Santiago,NaN


```SQL
SELECT
    COALESCE(c.ciudad, pr.ciudad) AS total_ciudad,
    pr.descuento
FROM clientes c
FULL OUTER JOIN promociones pr ON c.ciudad = pr.ciudad;
```